### This notebook should be executed from the main path of the Yolo5 repository:

i.e. from here:
https://github.com/ultralytics/yolov5

In fact instead of running this notebook, you can just run this one:
https://github.com/monaj07/yolov5/blob/b85e87855823f740b10daa12e1e7b09839f353c0/yolo5_finetuning.ipynb,
which is the same as the current notebook.
The current notebook is here just as a reminder to not forget the above one.

In [ ]:
# !pip install PyYAML==5.3.1
# !pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2

In [ ]:
# import cv2

# def overlay_bbox(img, label, box_points):
#     H, W = img.shape[:-1]
#     p1, p2 = box_points
#     x1, y1 = p1['x'] * W, p1['y'] * H
#     x2, y2 = p2['x'] * W, p2['y'] * H

#     cv2.rectangle(img, 
#                   (int(x1), int(y1)),
#                   (int(x2), int(y2)),
#                   color=(0, 255, 0),
#                   thickness=2)
    
#     (label_width, label_height), _ = cv2.getTextSize(
#         label, 
#         fontFace=cv2.FONT_HERSHEY_PLAIN,
#         fontScale=1.75, 
#         thickness=2)

#     cv2.rectangle(img, 
#                   (int(x1), int(y1)),
#                   (int(x1 + label_width), int(y1 + label_height)),
#                   color=(0, 255, 0),
#                   thickness=cv2.FILLED)
    
#     cv2.putText(
#         img,
#         label,
#         org=(int(x1), int(y1 + label_height)),
#         fontFace=cv2.FONT_HERSHEY_PLAIN,
#         fontScale=1.75,
#         color=(255, 255, 255),
#         thickness=2
#     )

#     return img

# img_bbox = overlay_bbox(img, 
#                         train_data[0]['annotation'][0]['label'][0], 
#                         train_data[0]['annotation'][0]['points'])
# plt.imshow(img_bbox); plt.show()

# Train on a custom dataset

### Download the dataset

https://github.com/ciber-lab/pictor-ppe

#### Direct link to the dataset:

https://drive.google.com/drive/folders/1M8nzvcnAsEXwz81x18X_mGeqPztZBIvO?usp=sharing

#### Unzip the downloaded files

In [ ]:
# !unzip Images-20210810T232206Z-001.zip -d worker_helmet_vest_dataset
# !unzip Labels-20210810T234322Z-001.zip -d worker_helmet_vest_dataset/Labels

#### An example image

In [ ]:
img = Image.open('../worker_helmet_vest_dataset/Images/image_from_china(4182).jpg')
img = np.array(img)
plt.imshow(img)
plt.show()

### Convert the raw dataset to a Yolo5 friendly dataset

Read this section carefully:
https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data#2-create-labels

In [ ]:
import os
import re
import shutil
import tqdm

def create_dataset(path='../worker_helmet_vest_dataset'):
    """
    YOLO v5 requires the dataset to be in the darknet format. 
    Here's an outline of what it looks like:

    One txt with labels file per image
    One row per object
    Each row: class_index bbox_x_center bbox_y_center bbox_width bbox_height
    Box coordinates must be normalized between 0 and 1
    """
    with open(os.path.join(path, 'Labels', 'pictor_ppe_crowdsourced_approach-02_train.txt'), 'r') as f:
        train_set = f.readlines()
    with open(os.path.join(path, 'Labels', 'pictor_ppe_crowdsourced_approach-02_valid.txt'), 'r') as f:
        val_set = f.readlines()
    with open(os.path.join(path, 'Labels', 'pictor_ppe_crowdsourced_approach-02_test.txt'), 'r') as f:
        test_set = f.readlines()
        
    train_set = [re.sub('\t', ' ', line.strip()) for line in train_set]
    val_set = [re.sub('\t', ' ', line.strip()) for line in val_set]
    test_set = [re.sub('\t', ' ', line.strip()) for line in test_set]
    
    dataset_split = {}
    dataset_split['train'] = {
        re.findall(r'(image_from_china\(\d+\).jpg).*', line)[0]: 
            [
                [int(item) for item in instance.split(',')] 
                    for instance in re.findall(r'image_from_china\(\d+\).jpg (.*)', line)[0].split()
            ] 
        for line in train_set
    }
    #
    dataset_split['val'] = {
        re.findall(r'(image_from_china\(\d+\).jpg).*', line)[0]: 
            [
                [int(item) for item in instance.split(',')] 
                    for instance in re.findall(r'image_from_china\(\d+\).jpg (.*)', line)[0].split()
            ] 
        for line in val_set
    }
    #
    dataset_split['test'] = {
        re.findall(r'(image_from_china\(\d+\).jpg).*', line)[0]: 
            [
                [int(item) for item in instance.split(',')] 
                    for instance in re.findall(r'image_from_china\(\d+\).jpg (.*)', line)[0].split()
            ] 
        for line in test_set
    }    
    
    for split in ["train", "val", "test"]:
        image_path = os.path.join(path, 'images', split)
        os.makedirs(image_path, exist_ok=True)
        label_path = os.path.join(path, 'labels', split)
        os.makedirs(label_path, exist_ok=True)

        for filename, image_instances in tqdm.tqdm(dataset_split[split].items(), total=len(dataset_split[split]), desc=f'{split}_data loading'):
            record = {}

            full_filename = os.path.join(path, 'Images', filename)
            full_filename_dest = os.path.join(image_path, filename)
            
            img = Image.open(full_filename)
            img = np.array(img)
            height, width = img.shape[:2]
            shutil.copyfile(full_filename, full_filename_dest)

            with open(os.path.join(label_path, '{}.txt'.format(re.findall(r"(.+)\..+", filename)[0])), 'w') as f:
                for instance in image_instances:                  
                    
                    py = [instance[1]/height, instance[3]/height]
                    px = [instance[0]/width, instance[2]/width]
                                        
                    bbox_width = px[1] - px[0]
                    bbox_height = py[1] - py[0]
                    f.write(
                      f"{instance[-1]} {px[0] + bbox_width / 2} {py[0] + bbox_height / 2} {bbox_width} {bbox_height}\n"
                    )

In [ ]:
create_dataset(path='../worker_helmet_vest_dataset')

### Fine-tuning
* img 320 - resize the images to 320x320 pixels (Larger is better, e.g. 640)
* batch 4 - 4 images per batch
* epochs 30 - train for 30 epochs
* data ./data/worker_helmet_vest_dataset.yaml - path to dataset config *(number of classes, path to the dataset, class labels)*
* cfg ./models/yolov5x_**4class**.yaml - model config
* weights yolov5x.pt - use pre-trained weights from the YOLOv5x model
* name yolov5x_clothing - name of our model
* cache - cache dataset images for faster training

In [ ]:
!python train.py --img 640 --batch 4 --epochs 30 \
    --data data/worker_helmet_vest_dataset.yaml \
    --cfg models/yolov5x_4class.yaml \
    --weights yolov5x.pt \
    --name yolov5x_worker_helmet_vest_dataset

#### Plot the training metric curves:

In [23]:
from utils.plots import plot_results

plot_results('./runs/train/yolov5x_worker_helmet_vest_dataset/results.csv');

### Inference

In [ ]:
!python detect.py --weights runs/train/yolov5x_worker_helmet_vest_dataset/weights/best.pt \
  --img 640 --conf 0.4 --source ../worker_helmet_vest_dataset/images/test/

In [ ]:
import torch
import torchvision


def load_image(path, resize=True):
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_AREA)
    return img

def show_grid(image_paths):
    images = [load_image(img) for img in image_paths]
    images = torch.as_tensor(images)
    images = images.permute(0, 3, 1, 2)
    print(images.shape)
    grid_img = torchvision.utils.make_grid(images, nrow=3)
    plt.figure(figsize=(24, 24))
    plt.imshow(grid_img.permute(1, 2, 0))
    plt.axis('off');
    plt.show()

In [ ]:
from glob import glob

img_paths = list(glob("runs/detect/exp/*.jpg"))[:9]
show_grid(img_paths)